In [17]:
include("Tools.jl");
include("GaugeFixing.jl")
include("KrylovTechnical.jl")
using DataFrames, CSV;

GiltTNR/GiltTNR2D_essentials.py:111: DeprecationWarning: invalid escape sequence '\ '
  """


Note that this notebook is not supposed to evaluate anything. It only loads, displays, and saves in a convinient format the precomputed results.

In [28]:
#change these values####
chi = 10               #
gilt_eps = 1e-4        #
########################

gilt_pars = Dict(
    "gilt_eps" => gilt_eps,
    "cg_chis" => collect(1:chi),
    "cg_eps" => 1e-10,
    "verbosity" => 0,
    "rotate" => false
)
gilt_pars_r = Dict(
    "gilt_eps" => gilt_eps,
    "cg_chis" => collect(1:chi),
    "cg_eps" => 1e-10,
    "verbosity" => 0,
    "rotate" => true
);


## Critical temperature with and without rotation (10 digits ):

In [29]:
id = gilt_pars_identifier(gilt_pars)
id_r = gilt_pars_identifier(gilt_pars_r)

crit_pars = deserialize("critical_temperatures/" * id * "__tol=1.0e-10.data")
crit_pars_r = deserialize("critical_temperatures/" * id_r * "__tol=1.0e-10.data")

println("             ", "relT lower bound   |  relT higher bound  | relT mean value")
println("no rotation  ", crit_pars[1], "  |  ", crit_pars[2], "  | ", (crit_pars[1] + crit_pars[2]) / 2)
println("   rotation  ", crit_pars_r[1], " |  ", crit_pars_r[2], " | ", (crit_pars_r[1] + crit_pars_r[2]) / 2)

             relT lower bound   |  relT higher bound  | relT mean value
no rotation  1.001277863159776  |  1.001277863234282  | 1.001277863197029
   rotation  1.0012784460932016 |  1.0012784461677073 | 1.0012784461304545


# Spectra of the jacobian at the approximate fixed point (no rotation)

In [30]:
id_no_rotation_tt = "eigensystems/" * gilt_pars_identifier(gilt_pars) * "__Z2_odd_sector=true_freeze_R=true_eigensystem.data"
id_no_rotation_tf = "eigensystems/" * gilt_pars_identifier(gilt_pars) * "__Z2_odd_sector=true_freeze_R=false_eigensystem.data"
id_no_rotation_ft = "eigensystems/" * gilt_pars_identifier(gilt_pars) * "__Z2_odd_sector=false_freeze_R=true_eigensystem.data"
id_no_rotation_ff = "eigensystems/" * gilt_pars_identifier(gilt_pars) * "__Z2_odd_sector=false_freeze_R=false_eigensystem.data"


spectrum_Z2T_FT = deserialize(id_no_rotation_tt)["eigensystem"][1]
spectrum_Z2T_FF = deserialize(id_no_rotation_tf)["eigensystem"][1]
spectrum_Z2F_FT = deserialize(id_no_rotation_ft)["eigensystem"][1]
spectrum_Z2F_FF = deserialize(id_no_rotation_ff)["eigensystem"][1];

specs = DataFrame()
specs[:, "Z2 odd sector included; R freezing: true"] = spectrum_Z2T_FT[1:30]
specs[:, "Z2 odd sector included; R freezing: false"] = spectrum_Z2T_FF[1:30]
specs[:, "Z2 odd sector excluded; R freezing: true"] = spectrum_Z2F_FT[1:30]
specs[:, "Z2 odd sector excluded; R freezing: false"] = spectrum_Z2F_FF[1:30]
specs[:, "Z2 odd sector included; Ising CFT prediction"] = 2 .^ (2 .- exact_spectrum(31)[2:end])
specs[:, "Z2 odd sector excluded; Ising CFT prediction"] = 2 .^ (2 .- exact_spectrum_even(31)[2:end])

CSV.write("export/spectra_chi=$chi.csv", specs)
specs

Row,Z2 odd sector included; R freezing: true,Z2 odd sector included; R freezing: false,Z2 odd sector excluded; R freezing: true,Z2 odd sector excluded; R freezing: false,Z2 odd sector included; Ising CFT prediction,Z2 odd sector excluded; Ising CFT prediction
,Complex…,Complex…,Complex…,Complex…,Float64,Float64
1,-4.06572+0.0im,-3.64961+0.0im,2.00021+0.0im,2.00716+0.0im,3.66802,2.0
2,2.00021+0.0im,2.00716+0.0im,0.955653+0.0im,0.940996+0.0im,2.0,1.0
3,-1.57801+0.0im,-1.55097+0.0im,0.934431+0.0im,0.938144+0.0im,1.83401,1.0
4,-1.55885+0.0im,-1.53948+0.0im,0.605999+0.0im,0.444071+0.0im,1.83401,1.0
5,0.955653+0.0im,0.940996+0.0im,0.441224+0.0im,-0.369765+0.0im,1.0,1.0
6,0.934431+0.0im,0.938144+0.0im,0.34372+0.0im,0.325835+0.0im,1.0,0.5
7,-0.762359+0.0im,-0.739821+0.0im,0.330111+0.086746im,0.313758+0.0im,1.0,0.5
8,-0.643836+0.0im,-0.529949+0.0im,0.330111-0.086746im,0.303789+0.0661615im,1.0,0.5
9,0.605999+0.0im,0.444071+0.0im,0.34042+0.0im,0.303789-0.0661615im,0.917004,0.5


## Spectrum of the jacobian after newton for Gilt with rotation

In [31]:
newton_result = deserialize("newton/" * gilt_pars_identifier(gilt_pars_r) * "__newton_result.data")
specs = DataFrame()
specs[:, "chi=$chi"] = newton_result["eigensystem"][1]

CSV.write("export/spectrum_chi=$(chi)_after_newton.csv", specs)
specs


Row,chi=10
,Complex…
1,2.00745+0.0im
2,-0.940972+0.0im
3,-0.936125+0.0im
4,0.444238+0.0im
5,-0.362702+0.0im
6,-2.78652e-7+0.318294im
7,-2.78652e-7-0.318294im
8,0.0627632+0.295995im
9,0.0627632-0.295995im
